In [8]:
from dotenv import load_dotenv
load_dotenv()

from typing import Annotated, List, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver

from chains import get_llm

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

llm = get_llm(api_key="OLLAMA")  
generate = prompt | llm

In [10]:
essay = ""

request = HumanMessage(
    content="Write an essay explaining the central theme of the count of Monte Cristo"
)

for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content


Certainly! Here is a five-paragraph essay exploring the central theme of "The Count of Monte Cristo" by Alexandre Dumas:

### The Central Theme of "The Count of Monte Cristo"

"The Count of Monte Cristo," also known as "The Count of Monte Christo," is a classic novel that delves into themes of justice, revenge, and redemption. At the heart of this narrative lies the theme of retribution and its consequences.

**Introduction**

The story revolves around Edmond Dantès, a young nobleman who becomes entangled in a plot against his life orchestrated by political rivals. After being unjustly imprisoned for 14 years on the island of Chateau d'If, he is released through the intervention of another character, Abbé Faria, who teaches him to read and write during his imprisonment. Upon his release, Dantès embarks on a journey filled with revenge against those who wronged him, as well as a quest for personal redemption.

**Body Paragraph 1**

One of the primary themes in "The Count of Monte Cristo

In [11]:
chat = ChatPromptTemplate.from_messages([
   ("system","You are a helpful AI Assistant with a sense of humor"),
   ("human","Hi how are you?"),
   ("ai","I am good. How can I help you?"),
   ("human","{input}")
])
print(chat)

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI Assistant with a sense of humor'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hi how are you?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='I am good. How can I help you?'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI Assistant with a sense of humor'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hi how are you?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='I am good. How can I help you?'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]

In [12]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [13]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Your essay provides a clear and insightful analysis of the central theme of "The Count of Monte Cristo." Here are some detailed critiques and recommendations for further development:

### Critique

1. **Introduction:**
   - The introduction could be more engaging to capture readers' attention from the start.
   - The opening sentence is good, but consider adding a brief background on the novel or its author to provide context.

2. **Body Paragraph 1:**
   - This paragraph effectively introduces and defines the theme of retribution.
   - However, it could benefit from more specific examples from the text to support your analysis.

3. **Body Paragraph 2:**
   - The paragraph is strong in discussing personal redemption but lacks depth in exploring how Edmond's experiences with Abbé Faria contribute to his understanding of justice and revenge.
   - Consider adding a quote or scene that illustrates this deeper exploration.

4. **Body Paragraph 3:**
   - This paragraph effectively delves int

In [14]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

Here is a revised version of the five-paragraph essay incorporating the critiques and recommendations provided:

### The Central Theme of "The Count of Monte Cristo"

"The Count of Monte Cristo," also known as "The Count of Monte Christo," by Alexandre Dumas, delves into themes of justice, revenge, and redemption. At the heart of this narrative is the theme of retribution and its consequences.

#### Introduction

In Dumas's classic novel, Edmond Dantès, a young nobleman unjustly imprisoned for 14 years, embarks on a transformative journey that explores the themes of justice, revenge, and personal redemption. The opening sentence captures the essence of the story: "I was not born to suffer in silence; I must have my revenge!" This quote sets the stage for Edmond's quest for retribution and his eventual realization of its true cost.

#### Body Paragraph 1

One of the primary themes in "The Count of Monte Cristo" is the concept of retribution. Edmond’s desire for vengeance stems from his 

In [15]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state["messages"])]}


async def reflection_node(state: State) -> State:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = await reflect.ainvoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}


builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")


def should_continue(state: State):
    if len(state["messages"]) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
memory = InMemorySaver()
graph = builder.compile(checkpointer=memory)

In [16]:
config = {"configurable": {"thread_id": "1"}}

In [17]:
async for event in graph.astream(
    {
        "messages": [
            HumanMessage(
                content="Generate an essay on the topicality of The Little Prince and its message in modern life"
            )
        ],
    },
    config,
):
    print(event)
    print("---")

{'generate': {'messages': [AIMessage(content='Certainly! Below is a 5-paragraph essay on the topicality of "The Little Prince" by Antoine de Saint-Exupéry and its enduring messages relevant to modern life:\n\n### The Topicality of The Little Prince and Its Messages in Modern Life\n\n"The Little Prince," written by Antoine de Saint-Exupéry, is a timeless tale that resonates deeply with readers across generations. While the story\'s setting is on a remote asteroid, it explores universal themes that remain pertinent today. One of its most significant messages concerns the importance of perspective and the value of simplicity in an increasingly complex world.\n\nIn "The Little Prince," the protagonist embarks on a journey to discover what he believes are the secrets of adulthood. However, his encounters with various adults reveal their superficiality and lack of genuine connection with children and nature. This narrative underscores how modern life often prioritizes materialism over meanin

In [18]:
state = graph.get_state(config)

In [19]:
ChatPromptTemplate.from_messages(state.values["messages"]).pretty_print()

================================ Human Message =================================

Generate an essay on the topicality of The Little Prince and its message in modern life

================================== Ai Message ==================================

Certainly! Below is a 5-paragraph essay on the topicality of "The Little Prince" by Antoine de Saint-Exupéry and its enduring messages relevant to modern life:

### The Topicality of The Little Prince and Its Messages in Modern Life

"The Little Prince," written by Antoine de Saint-Exupéry, is a timeless tale that resonates deeply with readers across generations. While the story's setting is on a remote asteroid, it explores universal themes that remain pertinent today. One of its most significant messages concerns the importance of perspective and the value of simplicity in an increasingly complex world.

In "The Little Prince," the protagonist embarks on a journey to discover what he believes are the secrets of adulthood. However, his 